In [ ]:
%pip install tensorflow[and-cuda] scikit-learn matplotlib seaborn pandas imbalanced-learn

Proyecto Aprendizaje Automático: Predicción del Alzheimer

Santiago Valdez Bocardo

Sebastián Arturo Jácome Herrera

Este código realiza un análisis y modelado predictivo del diagnóstico de Alzheimer utilizando Machine Learning y Deep Learning.

Pasos principales:
1. **Carga y exploración de datos**: Se analiza la estructura del dataset.
2. **Preprocesamiento**: Se eliminan columnas redundantes, se codifican variables categóricas y se escalan datos numéricos.
3. **Balanceo de clases**: Se aplican técnicas de Near Miss y SMOTE para abordar el desbalance en los datos.
4. **Entrenamiento de modelos**: Se prueban modelos de Decision Tree, Logistic Regression, SVM y Neural Network.
5. **Reducción de dimensionalidad**: Se aplica PCA para mejorar el desempeño de los modelos.
6. **Optimización del mejor modelo**: Se usa GridSearch para ajustar hiperparámetros en la red neuronal.
7. **Evaluación y visualización de resultados**: Se calculan métricas como Accuracy, Precision, Recall y F1-Score y se generan visualizaciones de los modelos.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn import tree
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import layers
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import itertools

In [ ]:
import warnings

warnings.filterwarnings('ignore')

Primeras filas

Mostrar las primeras filas del dataset para tener una idea general de su contenido

In [ ]:
file_path = "./alzheimers_prediction_dataset.csv"
df = pd.read_csv(file_path)
df.head(5)

# Conjunto de datos

Alzheimer’s Prediction Dataset (Global) es un conjunto de datos que contiene 74,283 registros provenientes de 20 países distintos. Este conjunto de datos es útil para modelos predictivos, estudios epidemiológicos e investigaciones sanitarias sobre la enfermedad de Alzheimer.



Atributos del Dataset


* Datos Demográficos:

Country: País de origen del individuo.

Age: Edad del individuo.

Gender: Género (Male/Female).

Education Level: Nivel educativo alcanzado (expresado en años de educación).

* Factores de Estilo de Vida:

Physical Activity Level: Nivel de actividad física (Bajo, Medio, Alto).

Smoking Status: Estado de tabaquismo (Nunca, Exfumador, Actual).

Alcohol Consumption: Frecuencia de consumo de alcohol (Nunca, Ocasionalmente, Regularmente).

Dietary Habits: Tipo de alimentación (Saludable, Promedio).

Air Pollution Exposure: Nivel de exposición a la contaminación del aire (Bajo, Medio, Alto).

Social Engagement Level: Nivel de participación en actividades sociales (Bajo, Medio, Alto).

Urban vs Rural Living: Lugar de residencia (Urbano/Rural).

* Factores Médicos:

BMI: Índice de Masa Corporal (IMC).

Diabetes: Presencia de diabetes (Sí/No).

Hypertension: Hipertensión arterial (Sí/No).

Cholesterol Level: Nivel de colesterol (Normal/Alto).

Family History of Alzheimer’s: Antecedentes familiares de Alzheimer (Sí/No).

Cognitive Test Score: Puntuación en pruebas cognitivas.

Depression Level: Nivel de depresión (Bajo, Medio, Alto).

Sleep Quality: Calidad del sueño (Buena, Regular, Pobre).


Stress Levels: Niveles de estrés (Bajo, Medio, Alto).

* Factores Genéticos y Económicos:

Genetic Risk Factor (APOE-ε4 allele): Presencia del alelo APOE-ε4 (Sí/No).

Income Level: Nivel de ingresos (Bajo, Medio, Alto).

Employment Status: Estado laboral (Empleado, Desempleado, Jubilado).

Marital Status: Estado civil (Soltero, Casado, Viudo).

* Variable Objetivo:

Alzheimer’s Diagnosis: Diagnóstico de Alzheimer, variable binaria con los valores:

0 (No): No se ha diagnosticado Alzheimer.

1 (Sí): Diagnóstico positivo de Alzheimer.




# Exploración de datos

Información General del Dataset

Análisis de la estructura general del dataset, incluyendo el número de columnas y tipos de datos

In [ ]:
df.info()

In [ ]:
df.shape

Valores Faltantes

In [ ]:
df.isnull().sum()

Datos Duplicados

In [ ]:
duplicated = df[df.duplicated()]
duplicated

In [ ]:
df.columns

In [ ]:
for i in range(len(df.columns)):
    ben = df.groupby(df.columns[i]).count()
    print("Column: ", ben.iloc[:, 0])
    print('\n')

There are 2 types of possible results:
* Yes with 30713 rows
* No with 43570 rows


Estadísticas generales

Visualización de estadísticas descriptivas para evaluar la distribución de las variables numéricas

In [ ]:
df.describe()

In [ ]:
df.info()

Distribución de variable objetivo

Esto ayuda a identificar si la variable objetivo está balanceada o no

In [ ]:
print(df.columns.tolist())

– ¿Cuántos datos tienes, cuántas filas, cuántas columnas?

Hay 1857075 datos, 74,283 filas y 25 columnas.

– ¿Hay datos faltantes?

El dataset no muestra datos faltantes

– ¿Hay contenido redundante o duplicado?

No se encuentran datos repetidos en el dataset, sin embargo algunas posibles columnas redundantes son:

* Country: No aporta información para la predicción individual.
* Cognitive Test Score: Correlacionada con la variable objetivo.
* Employment Status, Marital Status, Income Level, Urban vs Rural Living:
 Podrían estar cubiertas por otras variables como Social Engagement Level o Education Level.

– ¿Cuál es la distribución de los datos en la clase objetivo?

Alzheimer’s Diagnosis (Variable Objetivo):
* No (sin Alzheimer): 58.65%
* Sí (con Alzheimer): 41.35%

La distribución no está completamente balanceada, pero no es un caso extremo de desbalanceo, así que se puede dejar como está

– ¿Existen correlaciones entre las categorías?

De acuerdo con la matriz, no hay correlaciones significativas entre las variables numéricas

Age, Education Level, BMI, y Cognitive Test Score tienen valores de correlación cercanos a 0 entre sí, indicando que no hay relaciones lineales fuertes entre estas variables.


Dado que no hay una relación fuerte entre las variables numéricas, no parece haber redundancia dentro de ellas.

In [ ]:
df.drop(columns=[
    "Country",
    "Cognitive Test Score",
    "Employment Status",
    "Marital Status",
    "Income Level",
    "Urban vs Rural Living"
], inplace=True)


df_processed = df

label_cols = [
    "Smoking Status", "Alcohol Consumption", "Dietary Habits"
]

label_encoder = LabelEncoder()
for col in label_cols:
    df_processed[col] = label_encoder.fit_transform(df[col])

one_hot_cols = [
    "Gender", "Diabetes", "Hypertension", "Genetic Risk Factor (APOE-ε4 allele)", 
    "Alzheimer’s Diagnosis", "Family History of Alzheimer’s"
]

df_processed = pd.get_dummies(df_processed, columns = one_hot_cols, drop_first=True)

ordinal_cols = [
    "Physical Activity Level", "Cholesterol Level", "Depression Level", "Sleep Quality", 
    "Air Pollution Exposure", "Social Engagement Level", "Stress Levels"
]

ordinal_mappings = { 
    "Physical Activity Level": ["Low", "Medium", "High"],
    "Cholesterol Level": ["Normal", "High"],
    "Depression Level": ["Low", "Medium", "High",],
    "Sleep Quality": ["Poor", "Average", "Good"],
    "Air Pollution Exposure": ["Low", "Medium", "High"],
    "Social Engagement Level": ["Low", "Medium", "High"],
    "Stress Levels": ["Low", "Medium", "High"]
}

ordinal_encoder = OrdinalEncoder(categories=[ordinal_mappings[col] for col in ordinal_cols])
df_processed[ordinal_cols] = ordinal_encoder.fit_transform(df_processed[ordinal_cols])

In [ ]:
df_processed.info()

In [ ]:
target_column = df_processed['Alzheimer’s Diagnosis_Yes']

plt.figure(figsize=(6, 4))
sns.countplot(x=target_column, data=df_processed, palette="viridis")
plt.title("Distribución de Alzheimer’s Diagnosis")
plt.xlabel("Diagnóstico de Alzheimer")
plt.ylabel("Frecuencia")
plt.show()

Matriz de Correlación

Se genera la matriz para analizar las correlaciones entre variables numéricas y detectar posibles relaciones

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(df_processed.corr(), annot=True, fmt=".2%", mask=np.triu(np.ones_like(df_processed.corr())))
plt.title("Matriz de correlación de variables numéricas")
plt.show()

Separación de datos

Se separan los datos en entrenamiento y prueba en una proporción 80%-20%

In [ ]:
X = df_processed.drop(columns=["Alzheimer’s Diagnosis_Yes"])
y = df_processed["Alzheimer’s Diagnosis_Yes"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Escalado de variables numericas

Se normalizan variables numéricas usando StandardScaler o MinMaxScaler para mejorar el desempeño de los modelos

In [ ]:
numerical_columns = ["Age", "Education Level", "BMI"]
scaler = StandardScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

In [ ]:
print("\n--- Información después del preprocesamiento ---")
print(X_train.info())
print("\n--- Primeras Filas del Dataset Preprocesado ---")
print(X_train.head())

# Procesamiento

Aplicación de técnicas de balanceo

Random Over Sampling
Duplica algunos ejemplos de nuestra clase minoritaria para estar a la par de la mayoritaria

Random Under Sampling
Elimina ejemplos de la clase mayoritaria para estar a la par de la minoritaria

Near-miss
Reduce la cantidad de muestras de la clase mayoritaria

In [ ]:
# undersampling
nearmiss = NearMiss()
X_train_nm, y_train_nm = nearmiss.fit_resample(X_train, y_train)

SMOTE

Genera nuevas muestras sintéticas para la clase minoritaria

In [ ]:
# oversampling
smote = SMOTE()
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

Definición y entrenamiento de modelos

Se entrena y evalúa cada modelo con los datasets original, Near Miss y SMOTE

In [ ]:
models = {
    "Decision Tree": tree.DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC()
}

In [ ]:
results = []
for model_name, model in models.items():
    for dataset_name, X_tr, y_tr in zip(["Original", "Near Miss", "SMOTE"],
                                        [X_train, X_train_nm, X_train_sm],
                                        [y_train, y_train_nm, y_train_sm]):
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results.append([model_name, dataset_name, accuracy])

Red neuronal

Se implementa una red neuronal con capas densas y dropout para evitar sobreajuste

In [ ]:
def train_neural_network(X_train, y_train, X_test, y_test, dataset_name):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=0)

    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    results.append(["Neural Network", dataset_name, accuracy])

In [ ]:
train_neural_network(X_train, y_train, X_test, y_test, "Original")
train_neural_network(X_train_nm, y_train_nm, X_test, y_test, "Near Miss")
train_neural_network(X_train_sm, y_train_sm, X_test, y_test, "SMOTE")

In [ ]:
results_df = pd.DataFrame(results, columns=["Modelo", "Dataset", "Accuracy"])
print(results_df)

# Logistic Regression

In [ ]:
# Random Under Sampling
undersampler = RandomUnderSampler(random_state=42)
X_under, y_under = undersampler.fit_resample(X, y)
print(len(X_under), len(y_under))
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)
model = LogisticRegression(class_weight='balanced' , max_iter=1000 , random_state=42)
model.fit(X_train, y_train)
model.score(X_test, y_test)
results.append(["Logistic Regression", 'Random Under Sampling', model.score(X_test, y_test)])

In [ ]:
# Random Over Sampling
oversampler = RandomOverSampler(random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)
print(len(X_over), len(y_over))
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)
model = LogisticRegression(class_weight='balanced' , max_iter=1000 , random_state=42)
model.fit(X_train, y_train)
model.score(X_test, y_test)
results.append(["Logistic Regression", 'Random Over Sampling', model.score(X_test, y_test)])

# Decision Tree

In [ ]:
# Random Under Sampling
undersampler = RandomUnderSampler(random_state=42)
X_under, y_under = undersampler.fit_resample(X, y)
print(len(X_under), len(y_under))
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)
dt_under = tree.DecisionTreeClassifier()
dt_under = dt_under.fit(X_train, y_train)
dt_under.score(X_test, y_test)
results.append(["Decision Tree", 'Random Under Sampling', dt_under.score(X_test, y_test)])

In [ ]:
# Random Over Sampling
oversampler = RandomOverSampler(random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)
print(len(X_over), len(y_over))
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)
dt_over = tree.DecisionTreeClassifier()
dt_over = dt_over.fit(X_train, y_train)
dt_over.score(X_test, y_test)
results.append(["Decision Tree", 'Random Over Sampling', dt_over.score(X_test, y_test)])

# Redes Neuronales

Aquí continuación se usa el escalador MinMaxScaler, que en general nos dio mejores resultados. También, por alguna razón, nos dio algunos errores el intentar hacerlo con StandardScaler al principio.

In [ ]:
# MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_processed)
df_scaled = pd.DataFrame(scaled_data, columns=df_processed.columns)


In [ ]:
# Under Sampler
undersampler = RandomUnderSampler(random_state=42)
X_under, y_under = undersampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)
model = keras.Sequential([])
model.add(keras.Input(shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=16, epochs=10)
res_nn = model.evaluate(X_test, y_test)
print(res_nn)
train_accuracy = history.history['accuracy'][-1]
results.append(["Neural Network", 'Random Under Sampling', train_accuracy])

In [ ]:
# Over Sampler
oversampler = RandomOverSampler(random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)
model = keras.Sequential([])
model.add(keras.Input(shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=16, epochs=10)
res_nn = model.evaluate(X_test, y_test)
print(res_nn)
train_accuracy = history.history['accuracy'][-1]
results.append(["Neural Network", 'Random Over Sampling', train_accuracy])

# SVM

In [ ]:
# Random Under Sampling
undersampler = RandomUnderSampler(random_state=42)
X_under, y_under = undersampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)
svm_under = SVC(kernel='rbf')
svm_under.fit(X_train, y_train)
svm_under.score(X_test, y_test)
results.append(["SVM", 'Random Under Sampling', svm_under.score(X_test, y_test)])

In [ ]:
# Random Over Sampling
oversampler = RandomOverSampler(random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)
svm_over = SVC(kernel='rbf')
svm_over.fit(X_train, y_train)
svm_over.score(X_test, y_test)
results.append(["SVM", 'Random Over Sampling', svm_over.score(X_test, y_test)])

In [ ]:
sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
results_df = pd.DataFrame(sorted_results, columns=["Modelo", "Algoritmo", "Accuracy"])
print(results_df)

Es importante denotar que puede que, debido a la cercanía entre precisiones, varíen los 3 mejores resultados.
En nuestra última ejecución, se pudo observar que los mejores resultados fueron:

Neural Network Over Sampling -> 0.7238
Neural Network Under Sampling -> 0.7237
Decision Tree Over Sampling -> 0.7168

Se usarán estos modelos para PCA

# PCA
Se usa PCA para reducir la dimensionalidad (cantidad de variables) y eliminar ruido en los datos

In [ ]:
X = df_scaled.drop("Alzheimer’s Diagnosis_Yes", axis=1)
y = df_scaled['Alzheimer’s Diagnosis_Yes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
reduced_exp_variance = list()

total=0
for n in range(len(explained_variance)):
    total += explained_variance[n]
    reduced_exp_variance.append(explained_variance[n])
    if(total>0.9):
        break
print("Número de componentes que forman el 90%: ", n)
print(reduced_exp_variance)


Visualización de la varianza explicada por los componentes principales

In [ ]:
background_color = "#ffffff"
fig = plt.figure(figsize=(20,7), facecolor=background_color)
plt.bar(range(n+1), reduced_exp_variance, alpha=0.5, align='center', label='Individual Explained Variance')
plt.ylabel('Explained Variance Ratio',  fontsize = 18)
plt.xlabel('Principal Components', fontsize = 18)
plt.title('Explained Variance Ratio vs Principal Components', fontsize = 25)
plt.show()

Se muestra que el valor se encuentra en los primeros 14 componentes, ya que explican la mayor parte de la varianza; se utilizarán para la aplicación del PCA

In [ ]:
# Neural Network Random Over Sampler (1er Lugar)
oversampler = RandomOverSampler(random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)

pca = PCA(n_components=n)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = keras.Sequential([])
model.add(keras.Input(shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=16, epochs=10)
res_nn = model.evaluate(X_test, y_test)
print(res_nn)
train_accuracy = history.history['accuracy'][-1]
print(train_accuracy)

In [ ]:
# Neural Network Random Under Sampler (2do Lugar)
undersampler = RandomUnderSampler(random_state=42)
X_under, y_under = undersampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)

pca = PCA(n_components=n)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = keras.Sequential([])
model.add(keras.Input(shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=16, epochs=10)
res_nn = model.evaluate(X_test, y_test)
print(res_nn)
train_accuracy = history.history['accuracy'][-1]
print(train_accuracy)

In [ ]:
# Decision Tree Random Over Sampling (3er Lugar)
oversampler = RandomOverSampler(random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)

pca = PCA(n_components=n)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

dt_over = tree.DecisionTreeClassifier()
dt_over = dt_over.fit(X_train, y_train)
dt_over.score(X_test, y_test)

# Modelado

En nuestro caso, incluso después de realizar el PCA. Se obtuvieron mejores resultados con el Neural Network utilizando Random Over Sampling.
Por lo que se hará el GridSearch con este modelo

Se decidió hacer Grid Search manualmente debido a  errores encontrados al intentar usar GridSearchCV con KerasClassifier.
GridSearchCV no funcionaba correctamente con KerasClassifier, generando el error '__sklearn_tags__ not found'. Esto nos forzó a realizarlo de forma manual, que es más tardado pero dará resultado.

In [ ]:
# Neural Network Random Over Sampler (GridSearchCV)
oversampler = RandomOverSampler(random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)
def create_model(neurons1, neurons2, neurons3, optimizer):
    model = keras.Sequential([])
    model.add(keras.Input(shape=(X_train.shape[1],)))
    model.add(keras.layers.Dense(neurons1, activation='relu'))
    model.add(keras.layers.Dense(neurons2, activation='relu'))
    model.add(keras.layers.Dense(neurons3, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

parameters = {
    'optimizer': ['adam', 'rmsprop'],
    'neurons1': [64, 32],
    'neurons2': [32, 16],
    'neurons3': [16, 8],
    'batch_size': [16, 32],
    'epochs': [5, 10],
}

param_combinations = list(itertools.product(
    parameters['optimizer'],
    parameters['neurons1'],
    parameters['neurons2'],
    parameters['neurons3'],
    parameters['batch_size'],
    parameters['epochs']
))
    
best_accuracy = 0
best_params = None

for optimizer, neurons1, neurons2, neurons3, batch_size, epochs in param_combinations:
    print(f"Entrenando con: optimizer={optimizer}, neurons layer 1={neurons1}, neurons layer 2={neurons2}, neurons layer 3={neurons3}, batch_size={batch_size}, epochs={epochs}")

    model = create_model(neurons1, neurons2, neurons3, optimizer)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_split=0.2)

    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.4f}")


    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = (optimizer, neurons1, neurons2, neurons3, batch_size, epochs)
        best_y_pred = y_pred

print("\nMejores hiperparámetros encontrados:")
print(f"Optimizer: {best_params[0]}, Neurons layer 1: {best_params[1]}, Neurons layer 2={best_params[2]}, Neurons layer 3={best_params[3]}, Batch Size: {best_params[4]}, Epochs: {best_params[5]}")
print(f"Mejor Accuracy: {best_accuracy:.4f}")

    

# Mejor modelo del GridSearch

A continuación se grafica visualmente la matriz de confusión

In [ ]:
cm = confusion_matrix(y_test, best_y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix Heatmap (SVM - Random Over Sampling)")
plt.show()

# Evaluación del modelo

Para no afectar valores que ya fueron ejecutados, para este paso se decidió entrenar los modelos nuevamente.

In [ ]:
# Neural Network Random Under Sampler (2do Lugar antes de GridSearch)
undersampler = RandomUnderSampler(random_state=42)
X_under, y_under = undersampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)
model = keras.Sequential([])
model.add(keras.Input(shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=16, epochs=10)
res_nn = model.evaluate(X_test, y_test)
print(res_nn)
train_accuracy = history.history['accuracy'][-1]

# Resultados Del GridSearch 

A continuación, se mostrará una gráfica que compara el modelo SVM, tanto con GridSearch, como sin este.

In [ ]:
results = [
    ["Neural Network Random Over Sampling con GridSearch", best_accuracy],
    ["Neural Network Random Under Sampling Sin GridSearch", train_accuracy]
]
df_results = pd.DataFrame(results, columns=["Model", "Accuracy"])

plt.figure(figsize=(6, 5))
sns.barplot(data=df_results, x="Model", y="Accuracy", palette="Blues")
plt.xlabel("Model Type")
plt.ylabel("Accuracy")
plt.title("Model Comparison: GridSearch Best vs SVM RBF")
plt.show()

## Análisis de evaluación 

1. Comparación entre métricas de modelos

La evaluación de los modelos se realizó con las métricas: Accuracy, Precision, Recall y F1-Score.

* Accuracy: El modelo con mejor accuracy es Neural Network con SMOTE (0.7228), seguido de SVM con datos originales (0.7195) y Logistic Regression con SMOTE (0.7144). Esto es sin contar las ejecuciones de los modelos anteriores, donde, en general, el modelo con mejor accuracy es Neural Network con SMOTE (0.7256)
* Precision: La precisión más alta es obtenida por Logistic Regression con SMOTE (0.6356) y Neural Network con datos originales (0.6593), y esto indica que cuando predicen la presencia de Alzheimer, son modelos confiables.
* Recall: El mejor recall lo obtuvo SVM con Near Miss (0.7610) y SVM con SMOTE (0.7501). Debido a esto se concluye que estos modelos identificaron mejor los casos positivos.
* F1-Score: La métrica que balancea precisión y recall tiene sus mejores valores en Neural Network con SMOTE (0.6721) y SVM con SMOTE (0.6818).


Con este análisis, podemos concluir que Neural Network con SMOTE es el mejor modelo, ya que logra el mejor balance entre todas las métricas, mostrando que no solamente se predice correctamente, sino que mantiene un buen recall y precisión a su vez.

2. Impacto del Balanceo de Clases
Se analizaron los efectos de las técnicas de balanceo Near Miss y SMOTE y se concluyó lo siguiente:

* SMOTE mejoró la mayoría de los modelos, logrando mejores valores en accuracy y recall al generar datos sintéticos en la clase minoritaria, sobre todo con Neural Network, como ya vimos.
* Near Miss redujo el rendimiento en general, ya que al eliminar datos puede haber perdido información relevante.


3. Interpretación de la matriz de confusión
La Matriz de Confusión del mejor modelo (Neural Network con SMOTE) mostró:

Altos valores en la diagonal principal, lo que indica que predice correctamente la mayoría de los casos. Hubieron errores pero no considerables para tomar el modelo como malo.

Hubieron menos falsos negativos comparado con otros modelos, lo cual es importante en caso de problemas médicos, ya que identificar correctamente casos positivos es lo más importante.

Con esto se puede concluir que Neural Network con SMOTE no solo obtuvo el mejor accuracy, sino que su matriz de confusión confirma que es el más confiable en la detección de Alzheimer.

# Interpretación de resultados

## Evaluación del impacto del balanceo de clases 

Los datos antes del balanceo de clases mostraron una pequeña desproporción con la variable objetivo, haciendo que pudiera afectar el rendimiento de los modelos. Debido a esto, se aplicaron técnicas de **Near Miss** (undersampling) y **SMOTE** (oversampling) para equilibrar las clases.
Los resultados obtenidos mostraron que:
- **SMOTE** mejoró el rendimiento en la mayoría de los modelos, especialmente en Neural Network y Logistic Regression.
- **Near Miss** generalmente disminuyó la precisión debido a la eliminación de datos relevantes, por lo que no funcionó mucho para este proyecto
- **SVM** y **Neural Network** se beneficiaron más del balanceo, obteniendo su mayor accuracy con SMOTE. Estos modelos se utilizaron para seguir con la evaluación.


## Importancia de las características y PCA

Se aplicó PCA para reducir la dimensionalidad y mejorar el desempeño de los modelos. PCA logra esto eliminando ruido y redundancias en las variables, lo que es útil para este proyecto.
Los obtenidos mostraron que:
- **Neural Network con SMOTE y PCA obtuvo un accuracy de 71.76%**, manteniendo un buen desempeño con menos características. Este fue, por mucho, el mejor accuracy de todos nuestros modelos
- **SVM y Logistic Regression mostraron una ligera disminución en accuracy tras haber aplicado PCA**, esto muestra que esta reducción de características no fue beneficiosa para estos modelos.
- En general, PCA nos permitió mejorar la eficiencia sin sacrificar demasiado el rendimiento, aunque no para todos los modelos.

## Conclusiones y visualización de resultados

Con este proyecto se pudo demostrar la importancia de realizar un pipeline completo de Machine Learning para el entrenamiento de modelos paraciertos proyectos, en este caso para un proyecto sobre Alzheimer. Se realizaron múltiples experimentos con técnicas de balanceo de clases, selección de características y optimización de hiperparámetros, para de esta forma poder construir el mejor modelo posible y obtener los mejores resultados.

1. **Neural Network con SMOTE y Grid Search es el mejor modelo**, alcanzando una accuracy superior a los otros métodos.
2. **SMOTE es la mejor estrategia de balanceo**, ya que permitió mejorar la precisión sin perder información relevante.
3. **PCA ayudó a reducir la complejidad del modelo**, aunque su impacto en la precisión varió según el algoritmo.
